# <font style="color:#008fff;">Machine Learning Modeling</font>
<hr>

In [1]:
import pandas as pd
import numpy as np
import time
import os
import sklearn
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
import sys
import random

#Disabling Warnings
warnings.filterwarnings('ignore')

# to make this notebook's output stable across runs
random.seed(42)

### Reading in preprocessed dataset

In [2]:
def loadDataset(file_name):
    df = pd.read_csv(file_name)
    return df

start_time= time.time()
df_train_preprocessed = loadDataset("Dataset/preprocessed_data.csv")
print("***Elapsed time to read csv files --- %s seconds ---***" % (time.time() - start_time))

***Elapsed time to read csv files --- 0.04900360107421875 seconds ---***


In [3]:
df_train_preprocessed.head(10)

,url_vect,is_gov_tld,who_is,https,profanity_score_prob,url_len_scaled,js_len_scaled,js_obf_len_scaled,label
0,0.044,0,0,0,0.901780,0.152542,0.842993,0.663632,1
1,0.005,0,1,1,0.001813,0.593220,0.097764,0.000000,0
2,0.033,0,0,0,0.965517,0.864407,0.727081,0.471829,1
3,0.046,0,1,1,0.049674,0.271186,0.038052,0.000000,0
4,0.046,0,0,0,0.955261,0.525424,0.600632,0.421721,1
5,0.017,0,1,1,0.006906,0.745763,0.155719,0.000000,0
6,0.046,0,0,0,0.925228,0.186441,0.492097,0.240813,1
7,0.524,0,0,0,0.989416,0.237288,0.393045,0.000000,1
8,0.046,0,0,0,0.896943,0.254237,0.715490,0.319687,1
9,0.046,0,0,1,0.007933,0.271186,0.128205,0.000000,0


### Getting X and y variables from our dataset

In [6]:
X = df_train_preprocessed.drop('label', axis=1)
y = df_train_preprocessed['label']

## <font style="color:#008fff;">K-Nearest Neighbors</font>

### Building KNN Model for the FULL feature set:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_full = KNeighborsClassifier(n_neighbors=3)
knn_full.fit(X, y)